In [2]:
import os,sys,warnings,click
import h5py, pandas
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
np.set_printoptions(legacy='1.25')
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit,fsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 24
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 6
mpl.rcParams['errorbar.capsize'] = 6
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"
import util as yu
yu.flag_fast=False

enss=['b','c','d']
ens2full={'a24':'cA211.53.24','a':'cA2.09.48','b':'cB211.072.64','c':'cC211.060.80','d':'cD211.054.96','e':'cE211.044.112'}
ens2label={'a24':'A24','a':'A48','b':'B64','c':'C80','d':'D96','e':'E112'}
ens2a={'a24':0.0908,'a':0.0938,'b':0.07957,'c':0.06821,'d':0.05692,'e':0.04892} # fm
ens2N={'a24':24,'a':48,'b':64,'c':80,'d':96,'e':112}
ens2N_T={'a24':24*2,'a':48*2,'b':64*2,'c':80*2,'d':96*2,'e':112*2}

hbarc = 1/197.3
ens2aInv={ens:1/(ens2a[ens]*hbarc) for ens in enss} # MeV

def find_t_cloest(ens,t):
    return round(t/ens2a[ens])

In [31]:
path='dataPrepare/cB211.072.64/data_aux/cfgs_run'
with open(path,'r') as f:
    cfgs=f.read().splitlines()

basepath='/p/project1/ngff/li47/code/scratch/run/02_discNJN_1D_run2/cB211.072.64/data_avgsrc/'
c3pt_dat=[]; c2pt_dat=[]
for icfg,cfg in enumerate(cfgs):
    print(f'{icfg}/{len(cfgs)}',end='                    \r')
    path=f'{basepath}{cfg}/discNJN_jg;stout.h5'
    with h5py.File(path) as f:
        moms_all=f['moms'][:]
        dic={}
        for i,mom in enumerate(moms_all):
            dic[tuple(mom)]=i
        moms=np.array([mom for mom in moms_all if (mom[3]**2+mom[4]**2+mom[5]**2==2)])
        inds_moms=[dic[tuple(mom)] for mom in moms]
        
        key='N_N_jg;stout20_6'
        t=f['data'][key][:,inds_moms,:,:]
        c3pt_dat.append(t)
        
    path=f'{basepath}{cfg}/N.h5'
    with h5py.File(path) as f:
        moms_all=f['moms'][:]
        dic={}
        for i,mom in enumerate(moms_all):
            dic[tuple(mom)]=i
        moms=np.array([mom for mom in moms_all if (mom[0]**2+mom[1]**2+mom[2]**2==2)])
        inds_moms=[dic[tuple(mom)] for mom in moms]
        
        t=f['data/N_N'][:,inds_moms]
        c2pt_dat.append(t)
c2pt_dat=np.array(c2pt_dat)
c3pt_dat=np.array(c3pt_dat)
print(end='                             \r')
print(c2pt_dat.shape)
print(c3pt_dat.shape)

(725, 37, 12)                
(725, 7, 12, 4, 10)
